In [1]:
# cell-1  
#load and clean the data (removing diacritics and unwanted text)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


log_file = 'SANAD.txt'
with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')

df = pd.read_csv('sanad/sanad.csv')
df.fillna('', inplace=True)

df['Category'] = df['Category'].astype('category')
# display(dfc['meter'].unique())

df['label'] = df['Category'].cat.codes #assign cat_value for each meter type
dftrain, dftest = train_test_split(df, test_size=0.20, random_state=42, stratify=df['label'])
ytrain = dftrain['label'].values.tolist()
ytest = dftest['label'].values.tolist()



max_sequence_length = 128
train_batch_size = 128
classes_num = len(df['Category'].unique())

display(classes_num)
display(len(df))
display(len(dftrain))
display(len(dftest))
df['Category'].unique()
display(df[:3])

2024-02-12 01:11:36.336194: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-12 01:11:36.462126: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-12 01:11:37.035542: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject


7

45500

36400

9100

,text,Category,label
0,"أعلن اتحاد كتاب وأدباء الإمارات فرع أبوظبي برنامج نشاطه خلال الأسبوع الجاري، وستكون البداية اليوم حيث يحتفي بيوم الشعر العالمي بأمسية شعرية موسيقية يتناوب فيها كل من الشاعرات زينب عامر الشاعرة والإعلامية الإماراتية صاحبة المجموعة الشعرية المطبوعة (ارتباك الشمس)؛ وشيخة المطيري الشاعرة الإماراتية صاحبة الديوانين المطبوعين (مرسى الوداد) و(للحنين بقية) فضلاً عن عشرات المشاركات في أمسيات ومهرجانات وتظاهرات ثقافية داخل الإمارات وخارجها؛ وفاطمة المعمري الشاعرة والكاتبة الإماراتية عضو اتحاد كتاب وأدباء الإمارات، وعضو رابطة أديبات الإمارات . وتأتي القراءات الشعرية على خلفية موسيقية حية تقدمها خريجة بيت العود العربي في مصر العازفة شيرين التهامي .تقام الأمسية في قاعة عبدالله عمران في مقر الاتحاد في معسكر آل نهيان في أبوظبي عند السابعة والنصف مساء .وفي مقره في المسرح الوطني في أبوظبي يستضيف الاتحاد عند السابعة والنصف مساء الاثنين الشاعر أحمد العسم، في أجزاء من نص طويل له بعنوان ""أمينة""، وسيوقع كتابه ""سوالف شوك"" وهو مجموعة نصوص بالمحكية .مساء الثلاثاء وبالتعاون مع هيئة أبوظبي للسياحة والثقافة ي...",Culture,0
1,بتوجيهات سمو الشيخ منصور بن زايد آل نهيان نائب رئيس مجلس الوزراء وزير شؤون الرئاسة شارك الأرشيف الوطني في الدورة ال49 لمعرض القاهرة الدولي للكتاب والتي افتتحتها أمس الأول إيناس عبد الدايم وزيرة الثقافة المصرية والدكتور عز الدين ميهوبي وزير الثقافة الجزائري وتستمر حتي العاشر من فبراير المقبل.وأكد حمد عبدالرحمن المدفع الأمين العام للمجلس الأعلى للاتحاد رئيس مجلس إدارة الأرشيف الوطني أهمية هذه المشاركة التي تتزامن مع عام زايد، حيث جاءت التوجيهات بأهمية نشر أهداف عام زايد الخير في أهم التظاهرات الثقافية في العالم العربي وليكون معرض القاهرة منصة هامة لتنفيذ مبادرات الأرشيف الوطني في عام زايد والتي تتضمن العديد من الفعاليات والمشاريع التي تؤرخ وتوثق مآثر ومناقب المغفور له الشيخ زايد بن سلطان آل نهيان، طيب الله ثراه. وقال محمد أحمد المر نائب رئيس مجلس إدارة الأرشيف الوطني إن الأرشيف الوطني حريص على المشاركة البناءة بمعرض القاهرة الدولي للكتاب؛ بوصفه تظاهرة ثقافية سنوية للمثقفين والباحثين والقراء، للتعريف بتاريخ دولة الإمارات ومؤسسها -المغفور له بإذن الله - الشيخ زايد بن سلطان آل نهيان، لا...,Culture,0
2,يقام على هامش الأيام معرض ذاكرة الأيام الذي درجت دائرة الثقافة والإعلام على تنظيمه بمصاحبة الأيام وهو معرض للصور الفوتوغرافية بعدسة المصور الفوتوغرافي محمود بنيان، وقد وثق فيه هذه السنة للدورة الماضية من الأيام، وقدم فيه 26 صورة لثلاثة عشر عرضاً بواقع صورتين لكل مسرحية، حرب النعل والرهان وقرموشة والسلوقي وعار الوقار وأصايل وحلوج . . إلخ، ويسعى المعرض إلى إبراز الديكور والسينوغرافيا والحركة على خشبة المسرح، وذلك باختيار زوايا عدة للصورة .ويقول محمود بنيان عن المعرض إنه يمثل ذاكرة حقيقية للأيام، نريد منها أن نوثق اللقطات الجميلة في هذه التظاهرة الوطنية الكبرى، ويمكن للصورة أن تعرف المشاهد على التطور الذي يحصل من مهرجان لآخر، كما أن اختيار اللقطات البارزة للفنانين الممثلين يشكل تاريخاً لهم، وذكرى جميلة تقول للمشاهد بأن هذا المسرحي المبدع مر من هنا، وأن الأيام كانت محطة بارزة في تاريخه .,Culture,0


In [2]:
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
import torch


models = ['faisalq/bert-base-arabic-wordpiece', 'faisalq/bert-base-arabic-senpiece',
          'faisalq/bert-base-arabic-bbpe']


for model_name in models:
    for i in range(3):
        print(f'{model_name}, try:{i}')
        
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = BertForSequenceClassification.from_pretrained(model_name,
                                                              num_labels=classes_num).to('cuda')                                                 
                                                             
        
        xtrain = tokenizer(
            text=dftrain['text'].tolist(),
            add_special_tokens=True,
            max_length = max_sequence_length,
            truncation=True,
            padding='max_length', 
            return_tensors='pt',
            return_token_type_ids = False,
            return_attention_mask = True,
            verbose = True)
        
        
        xtest = tokenizer(
            text=dftest['text'].tolist(),
            add_special_tokens=True,
            max_length = max_sequence_length,
            truncation=True,
            padding='max_length', 
            return_tensors='pt',
            return_token_type_ids = False,
            return_attention_mask = True,
            verbose = True)
        
        
        class NewGroupDataset(torch.utils.data.Dataset):
            def __init__(self, encodings, labels):
                self.encodings = encodings
                self.labels = labels
        
            def __getitem__(self, idx):
                item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
                item['label'] = torch.tensor([self.labels[idx]])
                return item
        
            def __len__(self):
                return len(self.labels)
        
        train_ds = NewGroupDataset(xtrain, ytrain)
        test_ds = NewGroupDataset(xtest, ytest)
        
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)    
            acc = accuracy_score(labels, predictions)        
            f1 = f1_score(labels, predictions, average='macro')   
            with open(log_file, 'a') as f:
                f.write(f'{model_name},{acc},{f1}\n')
            return {'accuracy': acc, 'f1_score': f1}
        
        
        
        
        epochs = 25
        save_steps = 10000 #save checkpoint every 10000 steps
        batch_size = 256
        
        training_args = TrainingArguments(
            output_dir = 'bert_cls/',
            overwrite_output_dir=True,
            num_train_epochs = epochs,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            save_steps = save_steps,
            save_total_limit = 5, #only save the last 5 checkpoints
            fp16=True,
            learning_rate = 5e-5,  # 5e-5 is the default
            logging_steps = 100, #50_000
            evaluation_strategy = 'epoch',
            # evaluate_during_training = True,
            eval_steps = 100
            
        )
        
        trainer = Trainer(
            model = model,
            args = training_args,
            # data_collator=data_collator,
            train_dataset=train_ds,
            eval_dataset=test_ds,
            compute_metrics = compute_metrics
        )
        
        
        # trainer.train(resume_from_checkpoint=True)
        trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results.to_csv('sanad_results.csv')
display(best_results)

faisalq/bert-base-arabic-wordpiece, try:0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at faisalq/bert-base-arabic-wordpiece and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.209900,0.071075,0.978571,0.978604
2,0.058300,0.065352,0.981209,0.981227
3,0.020700,0.083161,0.980549,0.980502
4,0.017600,0.082186,0.983736,0.983728
5,0.007600,0.087516,0.984176,0.984184
6,0.006300,0.094298,0.983626,0.983643
7,0.003700,0.105653,0.981758,0.981758
8,0.003400,0.112045,0.983077,0.983064
9,0.001300,0.102031,0.983077,0.983066
10,0.000600,0.100102,0.984725,0.984723


faisalq/bert-base-arabic-wordpiece, try:1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at faisalq/bert-base-arabic-wordpiece and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.192500,0.064695,0.980330,0.980348
2,0.051000,0.055186,0.983626,0.983636
3,0.020800,0.065042,0.984066,0.984073
4,0.014700,0.068867,0.983956,0.983938
5,0.005400,0.081378,0.984396,0.984411
6,0.005400,0.083637,0.984945,0.984922
7,0.002100,0.083544,0.985604,0.985605
8,0.002100,0.097840,0.983516,0.983514
9,0.001600,0.090923,0.985385,0.985377
10,0.000800,0.087381,0.985604,0.985600


faisalq/bert-base-arabic-bbpe, try:2


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at faisalq/bert-base-arabic-bbpe and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.192500,0.064695,0.980330,0.980348
2,0.051000,0.055186,0.983626,0.983636
3,0.020800,0.065042,0.984066,0.984073
4,0.014700,0.068867,0.983956,0.983938
5,0.005400,0.081378,0.984396,0.984411
6,0.005400,0.083637,0.984945,0.984922
7,0.002100,0.083544,0.985604,0.985605
8,0.002100,0.097840,0.983516,0.983514
9,0.001600,0.090923,0.985385,0.985377
10,0.000800,0.087381,0.985604,0.985600


,Model,Accuracy,F1
0,faisalq/bert-base-arabic-bbpe,0.986044,0.986042
1,faisalq/bert-base-arabic-bbpe,0.986044,0.986042
2,faisalq/bert-base-arabic-bbpe,0.986044,0.986042
3,faisalq/bert-base-arabic-senpiece,0.985714,0.985712
4,faisalq/bert-base-arabic-senpiece,0.985714,0.985712
5,faisalq/bert-base-arabic-senpiece,0.985714,0.985712
6,faisalq/bert-base-arabic-wordpiece,0.986044,0.986041
7,faisalq/bert-base-arabic-wordpiece,0.986044,0.986041
